In [55]:
import yaml
import spotipy
import urllib as url
import random
import pickle
from spotipy.oauth2 import SpotifyOAuth

In [2]:
creds = yaml.load(open('/Users/daverappeport/.spotify/creds.yml','r'), Loader=yaml.FullLoader)
redirect_url="http://localhost:9001/callback"
scope = ["user-library-read", "playlist-modify-private","playlist-modify-public"]

In [3]:
auth_manager=SpotifyOAuth(client_id=creds.get('client_id'), client_secret=creds.get('client_secret'), redirect_uri=redirect_url, scope=scope)
sp=spotipy.Spotify(auth_manager=auth_manager)

In [4]:
results = sp.current_user_saved_tracks()

In [5]:
results.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [6]:
results.get('items')[0].get('track').keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [7]:
user=sp.current_user()

In [8]:
for plist in sp.user_playlists(user=user['id']).get('items'):
    print(plist.get('name'))

My Peloton Music by RaveDapeport
Golden Hour
Rapp Haus
Ghost Town
Asian Dinner
Vibes
Cruise
The Long and Winding Road
good shit
Workout
Starred
Gimme Some More
Liked from Radio
GDTG


In [9]:
running_seeds_url="https://open.spotify.com/playlist/5NrLk0qUyICYEG8cZWK7SL"
res_playlist_id=url.parse.urlparse(running_seeds_url).path.split('/')[2]

In [10]:
running_recs_url="https://open.spotify.com/playlist/3Elnf5eDmqXA9dC52Dr9UJ"
rec_playlist_id=url.parse.urlparse(running_recs_url).path.split('/')[2]

In [11]:
seed_tracks = sp.playlist_items(playlist_id=res_playlist_id)

In [12]:
seed_track_ids = [track.get('track').get('id') for track in seed_tracks.get('items')]

In [13]:
seed_track_ids

['6rx7O4uhp7mPfNvq9FCkos',
 '7MkZtFx58ZUBF4RqwOBQwk',
 '6Nn74LbI9LzxgHci3rDQ6w',
 '5v4vZzXgcvSzFk9Ftlc1OP',
 '3WqifovLWUQICYOAad0aNP']

In [14]:
res = sp.recommendations(seed_tracks=seed_track_ids)
len(res.get('tracks'))

20

In [15]:
rec_track_ids = [track.get('id') for track in res.get('tracks')]

In [19]:
for track in res.get('tracks'):
    print(track.get('name'), ',', track.get('artists')[0].get('name'))

The Difference , Flume
Awaken , Big Wild
Highland Grace , Hiss Golden Messenger
Cream On Chrome - Single Edit , Ratatat
Temptation , SIDEPIECE
Catching Plays , Destructo
Little Walter Rides Again , Medeski Scofield Martin & Wood
Taking it Easy - Radio Edit , GoodLuck
Tearing Me Up - Edit , Bob Moses
It All Feels Right , Washed Out
The Trouble With Us , Marcus Marr
Dímelo , Snakehips
Somewhere Else (with BabyJake) , Louis The Child
In Spite of Ourselves (feat. Iris DeMent) , John Prine
The Brae , Yumi Zouma
50 Ways To Leave Your Lover , G. Love & Special Sauce
Nobody , Gorgon City
Roses Are Free , Ween
Tu Tu Tu (That's Why We) , Galantis
Stay For It , RL Grime


In [20]:
sp.playlist_replace_items(playlist_id=rec_playlist_id, items=rec_track_ids)

{'snapshot_id': 'NCwzOTUyNGMxMzNhZTRmYWIxMzk2ODk0NGM3YWEzYmY2MzQxYjk2MDcx'}

### if you need to clear the playlist
sp.playlist_remove_all_occurrences_of_items(
    playlist_id=rec_playlist_id, 
    items=[track.get('track').get('id') for track in sp.playlist_items(playlist_id=rec_playlist_id).get('items')]
)

In [22]:
# audio features
sp.audio_features(tracks=rec_track_ids)[:3]

[{'danceability': 0.56,
  'energy': 0.865,
  'key': 1,
  'loudness': -1.148,
  'mode': 1,
  'speechiness': 0.0909,
  'acousticness': 0.255,
  'instrumentalness': 0.000153,
  'liveness': 0.083,
  'valence': 0.424,
  'tempo': 159.96,
  'type': 'audio_features',
  'id': '4nlvKIIetOWGIMyhjQXgOZ',
  'uri': 'spotify:track:4nlvKIIetOWGIMyhjQXgOZ',
  'track_href': 'https://api.spotify.com/v1/tracks/4nlvKIIetOWGIMyhjQXgOZ',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4nlvKIIetOWGIMyhjQXgOZ',
  'duration_ms': 139663,
  'time_signature': 4},
 {'danceability': 0.493,
  'energy': 0.344,
  'key': 2,
  'loudness': -8.876,
  'mode': 1,
  'speechiness': 0.0381,
  'acousticness': 0.752,
  'instrumentalness': 0.000499,
  'liveness': 0.303,
  'valence': 0.119,
  'tempo': 139.994,
  'type': 'audio_features',
  'id': '4oYzwOTWpP3JTyKrTHJtqw',
  'uri': 'spotify:track:4oYzwOTWpP3JTyKrTHJtqw',
  'track_href': 'https://api.spotify.com/v1/tracks/4oYzwOTWpP3JTyKrTHJtqw',
  'analysis_url': 'https:

In [32]:
#### let's get my 200 most recent saved tracks
saved_tracks = []

for n in range(0,4):
    tracks = sp.current_user_saved_tracks(limit=50, offset=n)
    saved_tracks = saved_tracks + tracks.get('items')

print(len(saved_tracks))

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])
dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])
dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])
dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])
200


In [23]:
??sp.current_user_saved_tracks

Signature: sp.current_user_saved_tracks(limit=20, offset=0, market=None)
Source:   
    def current_user_saved_tracks(self, limit=20, offset=0, market=None):
        """ Gets a list of the tracks saved in the current authorized user's
            "Your Music" library

            Parameters:
                - limit - the number of tracks to return
                - offset - the index of the first track to return
                - market - an ISO 3166-1 alpha-2 country code

        """
        return self._get("me/tracks", limit=limit, offset=offset, market=market)
File:      ~/venv/dlrappeport/lib/python3.8/site-packages/spotipy/client.py
Type:      method


In [33]:
# res.get('items')[0].get('track')
new_music = []

for item in saved_tracks:
    item = item.get('track')
    # print(item.get('name') + ';' + item.get('artists')[0].get('name'))
    
    seed_id = item.get('id')
    recs = sp.recommendations(seed_tracks=[seed_id])
    
    for track in recs.get('tracks'):
        new_music.append(track)

Moonwalk Away;GoldFish
Hey Jude;The Overton Berry Trio
Arcadia;Goose
Hot Hands;Darius
Roll Back - Single Mix;George FitzGerald
Electrify Me;ZHU
NUMB;Hayden James
Lady Madonna;Fats Domino
Blueberry Hill;Fats Domino
Marissa;Mike Gordon
I Know You Rider - Live;The National
Mahalla;Carolina Chocolate Drops
Go Go Round This World!;Fishmans
Hook;Blues Traveler
Waiting All Night - Live;Phish
The Wilhelm Scream;The Bamboos
Bees;Caribou
Rudie Can't Fail - Remastered;The Clash
Kiss With A Fist;Florence + The Machine
Antigone - Yotto Remix;Joris Voorn
Águas De Março;Antônio Carlos Jobim
I Like It;DeBarge
Cleopatra;The Lumineers
Kick It to Me;Sammy Rae & The Friends
Rocky Raccoon;Scary Pockets
Say My Name;Matvey Emerson
Wash My Soul;Chris Malinchak
In The Air Tonight - Croatia Squad Remix;Nora En Pure
I Wanna Dance With Somebody (Who Loves Me);Nora Van Elken
I'm On Fire;ConKi
Everybody’s Gotta Live;Love
You're the One;The Black Keys
The Choice Is Yours (Revisited);Black Sheep
Expensive Shit;Fela K

In [34]:
# and now we have 4k new tracks
print(len(new_music))

new_music[0]

4000


{'album': {'album_type': 'SINGLE',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5UVzX8pQe6bb5ueNdfViih'},
    'href': 'https://api.spotify.com/v1/artists/5UVzX8pQe6bb5ueNdfViih',
    'id': '5UVzX8pQe6bb5ueNdfViih',
    'name': 'Chris Malinchak',
    'type': 'artist',
    'uri': 'spotify:artist:5UVzX8pQe6bb5ueNdfViih'}],
  'available_markets': ['AE',
   'AG',
   'AR',
   'BB',
   'BD',
   'BH',
   'BN',
   'BO',
   'BS',
   'BT',
   'BZ',
   'CL',
   'CO',
   'CR',
   'CW',
   'CY',
   'DM',
   'DO',
   'EC',
   'EG',
   'ES',
   'FM',
   'GB',
   'GD',
   'GR',
   'GT',
   'GY',
   'HK',
   'HN',
   'HT',
   'ID',
   'IE',
   'IN',
   'JM',
   'JO',
   'KH',
   'KN',
   'KR',
   'KW',
   'LA',
   'LB',
   'LC',
   'LK',
   'MO',
   'MV',
   'MX',
   'MY',
   'NI',
   'NP',
   'OM',
   'PA',
   'PE',
   'PH',
   'PK',
   'PS',
   'PY',
   'QA',
   'SA',
   'SG',
   'SR',
   'SV',
   'TH',
   'TL',
   'TR',
   'TT',
   'TW',
   'UY',
   'VC',
   'VN'],
  '

In [54]:
new_music_dict = {}

for m in new_music:
    new_music_dict[m.get('id')] = m
    
# get audio features
for n in range(0, int(len(new_music)/50)):
    features = sp.audio_features(tracks=[t.get('id') for t in new_music[n*50:(n+1)*50]])
    
    for f in features:
        new_music_dict[f.get('id')]['audio_features'] = f

In [61]:
k = random.choice(list(new_music_dict.keys()))
new_music_dict[k]

{'album': {'album_type': 'ALBUM',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0JbqyQSsMK7TavqlR0yNON'},
    'href': 'https://api.spotify.com/v1/artists/0JbqyQSsMK7TavqlR0yNON',
    'id': '0JbqyQSsMK7TavqlR0yNON',
    'name': 'Bobbi Humphrey',
    'type': 'artist',
    'uri': 'spotify:artist:0JbqyQSsMK7TavqlR0yNON'}],
  'available_markets': ['AD',
   'AE',
   'AG',
   'AL',
   'AM',
   'AO',
   'AR',
   'AT',
   'AU',
   'AZ',
   'BA',
   'BB',
   'BD',
   'BE',
   'BF',
   'BG',
   'BH',
   'BI',
   'BJ',
   'BN',
   'BO',
   'BR',
   'BS',
   'BT',
   'BW',
   'BY',
   'BZ',
   'CA',
   'CH',
   'CI',
   'CL',
   'CM',
   'CO',
   'CR',
   'CV',
   'CW',
   'CY',
   'CZ',
   'DE',
   'DJ',
   'DK',
   'DM',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'FI',
   'FJ',
   'FM',
   'FR',
   'GA',
   'GB',
   'GD',
   'GE',
   'GH',
   'GM',
   'GN',
   'GQ',
   'GR',
   'GT',
   'GW',
   'GY',
   'HK',
   'HN',
   'HR',
   'HT',
   'HU',
   'ID

In [63]:
# save spotify data to pickle file    
with open('new_music_dict.pickle', 'wb') as handle:
    pickle.dump(new_music_dict, handle)